In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
#from utils import get_openai_api_key
#openai_api_key = get_openai_api_key()
#os.environ["OPENAI_MODEL_NAME"] = 'wizardlm2:7b'
#os.environ["OPENAI_API_BASE"]='http://localhost:11434/v1'
#os.environ["OPENAI_API_KEY"]='NA'
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["LOCAL_EMBEDDINGS"]="http://localhost:11434/api/embeddings"
os.environ["LOCAL_EMBEDDINGS_MODEL"]="nomic-embed-text:v1.5"

### Agent - Busca no Contexto

In [4]:
from search.context import utils
from crewai_tools import tool

# Embeddings Model
embeddings_model = "nomic-embed-text:v1.5"
# Vectore Store path
path_books = "./vector-store/chroma/books"

@tool("Pesquisar em Livros")
def library_tool(query: str) -> str:
    """Retorna livros que possuem conteúdo relacionado com termo ou assunto buscado."""
    result = ""
    _, db = utils.retriver_context(embeddings_model=embeddings_model, path_books=path_books, vector_store='chroma')
    documents = db.similarity_search(query)
    for row in documents:
        page_content = row.page_content
        source = row.metadata['source']
        page = row.metadata['page']
        result += f"**Trecho de {source.replace('./books/import/','').replace('.pdf','')}** - página {page}\n"
        result += f"{page_content}\n\n"
    return result

In [5]:
context_agent = Agent(
    role="Pesquisador acurado de livros",
	goal="Pesquisar trechos de livros com relação com a query: '{query}'.",
	backstory=(
		"Você é um pesquisador e bibliotecário especializado em livros."
        "Seu idioma é o português."
		"Assim você retornará citações diretamente relacionadas com o "
        "assuntos do cliente e não faça suposições."
	),
	allow_delegation=False,
	verbose=True,
    tools=[library_tool],
)


qa_context_agent = Agent(
    role="Avaliador de relação de textos",
	goal="Com base em {query}, "
    	"avaliar se os trechos são diretamente relacionados.",
	backstory=(
		"Seu idioma é o português."
		"Você gosta do texto organizado texto base seguida do trecho da citação relacionada e "
        "a explicação simples da relação entre ambos."
	),
	allow_delegation=False,
	verbose=True
)

### Task - Busca no contexto

In [6]:
context_search = Task(
    description=(
        "1. Use a ferramenta Pesquisar em Livros enviando a query com o texto abaixo:\n"
        "{query}\n"
        "2. Encontrar trechos no retorno diretamente relacionados.\n"
        "3. Retorn apenas trechos diretamente relacionados."
        "4. A pesquisa deve ser feita em português.\n"
    ),
    expected_output="Citações dos trechos diretamente relacionados "
        "dos livros e suas fontes. Retorne em português",
    agent=context_agent,
)

In [7]:
context_validation = Task(
    description=(
        "1. Use como base o texto {query} \n"
        "2. Selecione os trechos de livros mais relevantes diretamente relacionados com o texto base.\n"
        "3. Retorne o texto base e a citação dos trechos relacionados.\n"
        "4. Faça uma breve explicação da relação entre o texto base e a citação.\n"
        "5. Responda em português.\n"
    ),
    expected_output="O texto base, as citações diretamente relacionadas e a explicação da relação entre ambos.",
    agent=qa_context_agent,
    context_agent=[qa_context_agent],
)

In [8]:
crew = Crew(
  agents=[context_agent, qa_context_agent],
  tasks=[context_search, context_validation],
  verbose=2,
  memory=False
)

In [9]:
inputs = {
    "query": """
27 O mais importante é que vocês vivam em sua comunidade de maneira digna das boas-novas de Cristo. Então, quando eu for vê-los novamente, ou mesmo quando ouvir a seu respeito, saberei que estão firmes e unidos em um só espírito e em um só propósito, lutando juntos pela fé que é proclamada nas boas-novas. 
28 Não se deixem intimidar por aqueles que se opõem a vocês. Isso é um sinal de Deus de que eles serão destruídos, e vocês serão salvos. 
29 Pois vocês receberam o privilégio não apenas de crer em Cristo, mas também de sofrer por ele. 
30 Estamos juntos nesta luta. Vocês viram as dificuldades que enfrentei no passado e sabem que elas ainda não terminaram.
Filipenses 1:27-30
"""
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Pesquisador acurado de livros
 [INFO]: == Starting Task: 1. Use a ferramenta Pesquisar em Livros enviando a query com o texto abaixo:

27 O mais importante é que vocês vivam em sua comunidade de maneira digna das boas-novas de Cristo. Então, quando eu for vê-los novamente, ou mesmo quando ouvir a seu respeito, saberei que estão firmes e unidos em um só espírito e em um só propósito, lutando juntos pela fé que é proclamada nas boas-novas. 
28 Não se deixem intimidar por aqueles que se opõem a vocês. Isso é um sinal de Deus de que eles serão destruídos, e vocês serão salvos. 
29 Pois vocês receberam o privilégio não apenas de crer em Cristo, mas também de sofrer por ele. 
30 Estamos juntos nesta luta. Vocês viram as dificuldades que enfrentei no passado e sabem que elas ainda não terminaram.
Filipenses 1:27-30

2. Encontrar trechos no retorno diretamente relacionados.
3. Retorn apenas trechos diretamente relacionados.4. A pesquisa deve ser feita em português.


In [10]:
from IPython.display import display, Markdown
Markdown(result)

O texto base e as citações selecionadas estão diretamente relacionados, pois abordam a importância de viver em conformidade com as boas-novas de Cristo, não se deixar intimidar por opositores, crer em Cristo e lutar pela fé proclamada de forma genuína e amorosa. A relação entre eles está na ênfase em viver uma vida de fé e testemunho cristão, mesmo diante de adversidades e desafios, buscando sempre propagar o evangelho com amor e autenticidade.